In [20]:

%%writefile search_phonebook.cu

#include <bits/stdc++.h>
#include <cuda.h>
#include <cuda_runtime.h>
using namespace std;

/* -------------------- STRUCT -------------------- */
struct Contact {
    char name[50];
    char number[50];
};

/* ---------------- DEVICE FUNCTION ---------------- */
__device__ bool checkSubstring(char* text, char* pattern, int patLen) {
    for (int i = 0; text[i] != '\0'; i++) {
        int j = 0;
        while (text[i + j] != '\0' &&
               pattern[j] != '\0' &&
               text[i + j] == pattern[j]) {
            j++;
        }
        if (j == patLen) {
            return true;
        }
    }
    return false;
}

/* ------------------- KERNEL --------------------- */
__global__ void searchPhonebook(Contact* phonebook,
                                int totalContacts,
                                char* searchName,
                                int searchLen) {

    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < totalContacts) {
        if (checkSubstring(phonebook[idx].name, searchName, searchLen)) {
            printf("%s %s\n",
                   phonebook[idx].name,
                   phonebook[idx].number);
        }
    }
}

/* -------------------- MAIN ---------------------- */
int main(int argc, char* argv[]) {

    if (argc != 3) {
        cerr << "Usage: " << argv[0]
             << " <search_name> <threads_per_block>\n";
        return 1;
    }

    string search_name = argv[1];
    int threadsPerBlock = atoi(argv[2]);

    /* ----------- FILE READING (CPU) -------------- */
    string file_name = "/content/drive/MyDrive/PP labtest_dataset1.txt";
    ifstream file(file_name);

    if (!file.is_open()) {
        cerr << "Error opening file!\n";
        return 1;
    }

    vector<Contact> phonebook;
    string line;
    Contact temp;

    while (getline(file, line)) {
        // Format: "name","phone_number"
        int pos = line.find(",");

        strcpy(temp.name, line.substr(1, pos - 2).c_str());
        strcpy(temp.number,
               line.substr(pos + 2, line.size() - pos - 4).c_str());

        phonebook.push_back(temp);
    }
    file.close();

    int totalContacts = phonebook.size();

    /* ------------- GPU MEMORY ------------------- */
    Contact* d_phonebook;
    cudaMalloc(&d_phonebook, totalContacts * sizeof(Contact));
    cudaMemcpy(d_phonebook,
               phonebook.data(),
               totalContacts * sizeof(Contact),
               cudaMemcpyHostToDevice);

    int searchLen = search_name.length();
    char* d_searchName;
    cudaMalloc(&d_searchName, searchLen + 1);
    cudaMemcpy(d_searchName,
               search_name.c_str(),
               searchLen + 1,
               cudaMemcpyHostToDevice);

    /* ------------- KERNEL LAUNCH ---------------- */
    int blocks =
        (totalContacts + threadsPerBlock - 1) / threadsPerBlock;

    searchPhonebook<<<blocks, threadsPerBlock>>>(
        d_phonebook,
        totalContacts,
        d_searchName,
        searchLen
    );

    cudaDeviceSynchronize();

    /* ------------- CLEANUP ---------------------- */
    cudaFree(d_phonebook);
    cudaFree(d_searchName);

    return 0;
}


Overwriting search_phonebook.cu


In [21]:
!nvcc -arch=sm_75 search_phonebook.cu -o search_phonebook

In [22]:
!time ./search_phonebook ZITU 2 > output.txt && sleep 2


real	0m8.132s
user	0m4.804s
sys	0m2.940s
